In [0]:
# Camada Silver - Sets - Magic: The Gathering
# Pipeline 100% PySpark DataFrame API
# Modularizado seguindo padrão Bronze/Silver

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import logging
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException



In [0]:
# =============================================================================
# FUNÇÕES UTILITÁRIAS
# =============================================================================
def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Secret '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Secret obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Required secret '{secret_name}' not configured")

def setup_unity_catalog(catalog, schema):
    try:
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
        spark.sql(f"USE CATALOG {catalog}")
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")
        spark.sql(f"USE SCHEMA {schema}")
        print(f"Schema {catalog}.{schema} criado ou já existente.")
        return True
    except Exception as e:
        print(f"Erro ao configurar Unity Catalog: {e}")
        return False

# =============================================================================
# EXTRAÇÃO
# =============================================================================
def extract_from_bronze(catalog, table_name):
    bronze_table = f"{catalog}.bronze.TB_BRONZE_SETS"
    df = spark.table(bronze_table)
    return df

# =============================================================================
# TRANSFORMAÇÃO
# =============================================================================
def transform_sets_silver(df):
    # Filtro temporal (últimos 5 anos)
    df = df.filter(col("DT_RELEASE") >= add_months(current_date(), -12*5))

    # Padronização de nomes (Title Case)
    df = df.withColumn("NME_SET", initcap(col("NME_SET")))
    df = df.withColumn("NME_TYPE", initcap(col("NME_TYPE")))
    df = df.withColumn("NME_BORDER", initcap(col("NME_BORDER")))
    df = df.withColumn("NME_MKM", initcap(col("NME_MKM")))
    df = df.withColumn("COD_SET", upper(col("COD_SET")))
    df = df.withColumn("COD_GATHERER", upper(col("COD_GATHERER")))
    df = df.withColumn("COD_MAGIC_CARDS_INFO", upper(col("COD_MAGIC_CARDS_INFO")))
    df = df.withColumn("COD_OLD", upper(col("COD_OLD")))

    # Limpeza e tratamento de nulos
    # Para descrições: substituir nulos/vazios por "NA"
    df = df.withColumn("NME_SET", when(col("NME_SET").isNull() | (col("NME_SET") == ""), lit("NA")).otherwise(trim(col("NME_SET"))))
    df = df.withColumn("NME_TYPE", when(col("NME_TYPE").isNull() | (col("NME_TYPE") == ""), lit("NA")).otherwise(trim(col("NME_TYPE"))))
    df = df.withColumn("NME_SOURCE", when(col("NME_SOURCE").isNull() | (col("NME_SOURCE") == ""), lit("NA")).otherwise(trim(col("NME_SOURCE"))))
    
    # Para códigos: substituir nulos/vazios por "NA"
    df = df.withColumn("COD_SET", when(col("COD_SET").isNull() | (col("COD_SET") == ""), lit("NA")).otherwise(trim(col("COD_SET"))))
    
    # Para números: substituir nulos por 0
    df = df.withColumn("RELEASE_YEAR", coalesce(col("RELEASE_YEAR"), lit(0)))
    df = df.withColumn("RELEASE_MONTH", coalesce(col("RELEASE_MONTH"), lit(0)))
    
    # Para booleanos: substituir nulos por false
    df = df.withColumn("FLG_ONLINE_ONLY", coalesce(col("FLG_ONLINE_ONLY"), lit(False)))

    # Conversão de datas
    df = df.withColumn("DT_RELEASE", col("DT_RELEASE").cast("date"))
    df = df.withColumn("DT_INGESTION", col("DT_INGESTION").cast("date"))

    # Particionamento (seguindo padrão da Bronze)
    df = df.withColumn("RELEASE_YEAR", year(col("DT_RELEASE")))
    df = df.withColumn("RELEASE_MONTH", month(col("DT_RELEASE")))

    # Seleção final de colunas - apenas as especificadas
    colunas_finais = [
        "COD_SET", "NME_SET", "NME_TYPE", "DT_RELEASE", 
        "FLG_ONLINE_ONLY", "NME_SOURCE", "RELEASE_YEAR", "RELEASE_MONTH"
    ]
    
    # Filtrar colunas que existem no DataFrame
    colunas_disponiveis = [c for c in colunas_finais if c in df.columns]
    
    df_final = df.select(*colunas_disponiveis)
    return df_final

# =============================================================================
# CARGA (WRITE/MERGE)
# =============================================================================
def delta_table_exists_and_schema_ok(spark, delta_path, df_final):
    try:
        delta_table = DeltaTable.forPath(spark, delta_path)
        current_schema = set([f.name for f in delta_table.toDF().schema.fields])
        new_schema = set([f.name for f in df_final.schema.fields])
        if current_schema != new_schema:
            return False, None
        return True, delta_table
    except Exception:
        return False, None

def load_to_silver_unity_incremental(df_final, catalog, schema, table_name, s3_silver_path):
    delta_path = f"s3://{s3_silver_path}/{table_name}"
    full_table_name = f"{catalog}.{schema}.{table_name}"
    print(f"Salvando dados em: {delta_path}")
    print("Qtd linhas df_final:", df_final.count())
    print("Colunas df_final:", df_final.columns)
    print("delta_path:", delta_path)

    exists, delta_table = delta_table_exists_and_schema_ok(spark, delta_path, df_final)
    if not exists:
        print("Tabela Delta não existe ou schema mudou. Salvando com overwrite e overwriteSchema=True.")
        try:
            df_final.write.format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .partitionBy("RELEASE_YEAR", "RELEASE_MONTH") \
                .save(delta_path)
            print("Write Delta concluído com sucesso!")
        except Exception as e:
            print("Erro no write Delta:", e)
            raise
    else:
        print("Tabela Delta já existe e schema é igual. Executando merge incremental por COD_SET.")
        count_antes = delta_table.toDF().count()
        df_final = df_final.dropDuplicates(["COD_SET"])
        update_cols = [c for c in df_final.columns if c != "COD_SET"]
        set_expr = {col: f"novo.{col}" for col in update_cols}
        merge_result = delta_table.alias("silver").merge(
            df_final.alias("novo"),
            "silver.COD_SET = novo.COD_SET"
        ).whenMatchedUpdate(set=set_expr) \
         .whenNotMatchedInsertAll() \
         .execute()
        count_depois = delta_table.toDF().count()
        print(f"Linhas antes do merge: {count_antes}")
        print(f"Linhas depois do merge: {count_depois}")
        print(f"Linhas adicionadas (diferença): {count_depois - count_antes}")
        print("Estatísticas do merge incremental:")
        for attr in [
            "insertedRowsCount", "updatedRowsCount", "deletedRowsCount",
            "copiedRowsCount", "sourceRowsCount", "outputRowsCount"
        ]:
            if hasattr(merge_result, attr):
                print(f"{attr}: {getattr(merge_result, attr)}")
    print("Dados salvos com sucesso na camada Silver!")

    # Criação/atualização da tabela no Unity Catalog
    try:
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")
        print(f"Schema {catalog}.{schema} criado ou já existente.")
    except Exception as e:
        print(f"Erro ao criar schema: {e}")
    try:
        from pyspark.sql.utils import AnalysisException
        # Verifica se a tabela já existe
        if spark.catalog.tableExists(full_table_name):
            # Pega o schema atual da tabela
            existing_schema = spark.table(full_table_name).schema
            # Função para comparar apenas nome e tipo das colunas (ignorando ordem e nullable)
            def schema_to_set(schema):
                return set((f.name.lower(), str(f.dataType).lower()) for f in schema.fields)
            if schema_to_set(existing_schema) == schema_to_set(df_final.schema):
                print(f"Tabela {full_table_name} já existe e schema é igual. Não será recriada.")
            else:
                print(f"Tabela {full_table_name} existe mas schema é diferente. Será recriada.")
                spark.sql(f"DROP TABLE IF EXISTS {full_table_name}")
                # Criar tabela com schema explícito para permitir particionamento
                spark.sql(f'''\
                    CREATE TABLE {full_table_name} (
                        COD_SET STRING,
                        NME_SET STRING,
                        NME_TYPE STRING,
                        DT_RELEASE DATE,
                        FLG_ONLINE_ONLY BOOLEAN,
                        NME_SOURCE STRING,
                        RELEASE_YEAR INT,
                        RELEASE_MONTH INT
                    )
                    USING DELTA
                    PARTITIONED BY (RELEASE_YEAR, RELEASE_MONTH)
                    LOCATION '{delta_path}'
                ''')
                print(f"Tabela Unity Catalog criada com particionamento: {full_table_name}")
        else:
            print(f"Tabela {full_table_name} não existe. Será criada.")
            # Criar tabela com schema explícito para permitir particionamento
            spark.sql(f'''\
                CREATE TABLE {full_table_name} (
                    COD_SET STRING,
                    NME_SET STRING,
                    NME_TYPE STRING,
                    DT_RELEASE DATE,
                    FLG_ONLINE_ONLY BOOLEAN,
                    NME_SOURCE STRING,
                    RELEASE_YEAR INT,
                    RELEASE_MONTH INT
                )
                USING DELTA
                PARTITIONED BY (RELEASE_YEAR, RELEASE_MONTH)
                LOCATION '{delta_path}'
            ''')
            print(f"Tabela Unity Catalog criada com particionamento: {full_table_name}")
    except AnalysisException as e:
        print(f"Erro ao criar tabela no Unity Catalog: {e}")



In [0]:
# =============================================================================
# PIPELINE PRINCIPAL
# =============================================================================

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
CATALOG_NAME = get_secret("catalog_name")
SCHEMA_NAME = "silver"
TABLE_NAME_BRONZE = "TB_BRONZE_SETS"
TABLE_NAME = "TB_REF_SILVER_SETS"
S3_BUCKET = get_secret("s3_bucket")
S3_SILVER_PREFIX = get_secret("s3_silver_prefix", "magic_the_gathering/silver")
S3_SILVER_PATH = f"{S3_BUCKET}/{S3_SILVER_PREFIX}"

setup_unity_catalog(CATALOG_NAME, SCHEMA_NAME)
df_bronze = extract_from_bronze(CATALOG_NAME, TABLE_NAME_BRONZE)
df_final = transform_sets_silver(df_bronze)
load_to_silver_unity_incremental(df_final, CATALOG_NAME, SCHEMA_NAME, TABLE_NAME, S3_SILVER_PATH)
